In [2]:

pip install timeit

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement timeit (from versions: none)
ERROR: No matching distribution found for timeit


In [1]:
import numpy as np
import pyarrow.parquet as pq
import plotly.graph_objects as go
import matplotlib.pylab as plt
from astropy.time import Time
from timeit import default_timer

# print(f"{np.__name__} --> version {np.__version__}")
# print(f"{pa.__name__} --> version {pa.__version__}")
!python -c "import numpy as xx; print(f'{xx.__name__} \t--> version {xx.__version__}')"
!python -c "import pyarrow as xx; print(f'{xx.__name__} --> version {xx.__version__}')"
!python -c "import plotly as xx; print(f'{xx.__name__} \t--> version {xx.__version__}')"
!python -c "import astropy as xx; print(f'{xx.__name__} --> version {xx.__version__}')"


numpy 	--> version 1.22.4
pyarrow --> version 9.0.0
plotly 	--> version 5.10.0
astropy --> version 5.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Universidad/Trabajo de Titulo/Modelos ML curvas de luz

# %cd /content/drive/MyDrive/Universidad/Trabajo de Titulo/Modelos ML curvas de luz#Aca va tu ruta del archivo en tu drive

ModuleNotFoundError: No module named 'google.colab'

In [3]:
parquet_file = pq.ParquetFile('../lightcurves_ts.parquet')
# parquet_file.schema_arrow.names # obtienes nombre columnas
# parquet_file.metadata.num_rows # numero de filas
# parquet_file.metadata.num_columns # nummero de columnas 

print("numero de columnas:", parquet_file.metadata.num_columns)
print("numero de filas (datos): ", parquet_file.metadata.num_rows)

numero de columnas: 113
numero de filas (datos):  13844917


In [4]:
# Descipcion de las columnas
# oid -- object id
# fid -- filter id, filtro o banda utilizada (verde,rojo,azul,etc)
# mjd -- medida de tiempo continua (Modified Julian date / Calendario Juliano Modificado)
# magpsf_corr -- magnitud psf correjida / posicion en el espacio, es el promedio de los pixeles de una deteccion
# sigmapsf_corr -- variacion de los pixeles de una deteccion
# nbad -- numero de bandas
# classALeRCE - clasificación que realiza ALeRCE en base a los parametros obtenidos (152 aprox) con el random forest de 2 niveles

# se divide el paquete en grupos --> se ocupa el primer grupo 0
data = parquet_file.read_row_group(0, ['oid', 'fid', 'mjd', 'magpsf_corr', 'sigmapsf_corr', 'nbad', 'classALeRCE']).to_pandas()
# data.shape
# data.head(50)

In [5]:
data.shape
data.head()

,oid,fid,mjd,magpsf_corr,sigmapsf_corr,nbad,classALeRCE
0,ZTF17aaacfxu,1,58863.138021,20.996371,0.444790,0,RRL
1,ZTF17aaacfxu,1,58867.179803,19.374605,0.175861,0,RRL
2,ZTF17aaacfxu,1,58871.170336,19.630466,0.123111,0,RRL
3,ZTF17aaacfxu,1,58880.146204,19.362001,0.103466,0,RRL
4,ZTF17aaacfxu,1,58849.131389,19.530453,0.078387,0,RRL


In [6]:
# muestra los eventos unicos, sin repeticiones
all_event = data.oid.unique()
# all_event

print('eventos totales',all_event.shape)

eventos totales (173693,)


In [8]:
data.oid.unique()

array(['ZTF17aaacfxu', 'ZTF17aaaczwj', 'ZTF17aaaemke', ...,
       'ZTF20abgdwuf', 'ZTF20abhagts', 'ZTF20abiqqow'], dtype=object)

In [34]:
def getStatistic(index,indBin,magnitud, amount_bins):
    #Analisis en base a la magnitud
    mag_ordered = magnitud[index]

    #Obtención de vectores con caracteristicas estadisticas
    vector_mean = np.array([np.nanmean(
        mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000  for i in range(amount_bins)])
    
    vector_max = np.array([np.nanmax(
        mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000 for i in range(amount_bins)])
    
    vector_min = np.array([np.nanmin(
        mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000 for i in range(amount_bins)])
    
    vector_std = np.array([np.nanstd(
        mag_ordered[indBin==(i+1)])  if (indBin==(i+1)).sum()>0 else -1000 for i in range(amount_bins)])

    return [vector_mean,vector_max,vector_min,vector_std]


def drawGraphic(bins,time,mag,indice,period):
    title_figure = f'Folded Time Serie with period T/{period}'
    #Grafico para ver la serie de tiempo 
    plt.figure(figsize=(12,4))
    plt.plot(time[indice], mag[indice])
    plt.scatter(time[indice], mag[indice])
    plt.ylabel("Magnitud")
    plt.xlabel("Time")
    plt.suptitle(title_figure, fontsize=22)
    plt.grid(axis='x', color='red', linestyle='-', linewidth=2)
    
    for bin in bins:
        plt.axvline(x=bin,color='red')

    # plt.grid(axis='y', color='gray', linestyle='dashed')
    plt.show()


def variantion_period_v2(array_periods,ts_time, ts_mag, amount_bins=10):
    
    tensor_data = []

    for index, period in enumerate(array_periods):

        #valor del periodo en decimales
        period_float = 1 / period

        #proceso de folding
        folded_time = ts_time % period_float

        #indices de los datos para encontrarlos luego del folding
        index_folded = np.argsort(folded_time)

        #Divido el periodo en bins o intervalos [0,n] para luego obtener datos estadisticos de cada bin
        bins = np.linspace(0,period_float,amount_bins+1)

        #Concentración de datos en cada bin
        concentration, bin_edges= np.histogram(folded_time[index_folded],bins=bins)
        
        #indices - Devuelve los índices de los bins a los que pertenece cada valor del folded_time
        index_bin = np.digitize(folded_time[index_folded],bins=bins)

        # print("bins: ",bins)
        # print("concentracion de datos en cada bin: ",concentration)

        #Visualizar grafico de cada variación del periodo
        # drawGraphic(bins,folded_time,ts_mag,index_folded,period)
    
        #Obtención de datos estadisticos de cada bin y formación del tensor
        vectorStatisticPeriod = np.asmatrix(getStatistic(index_folded,index_bin,ts_mag,amount_bins))
        tensor_data.append(vectorStatisticPeriod)

    return tensor_data


# funcion para graficar la serie de tiempo con link al sitio web de ALeRCE del evento
def GraphicLightCurve(fig,band,light_curve,show):
    mtcolor = [['g','green'],['r','red'],['b','blue']]
    idx=(light_curve['fid']==band).values
    fig.add_trace(
            go.Scatter(
                x = light_curve.mjd.values[idx],
                y = light_curve.magpsf_corr.values[idx],
                mode = 'markers', name=mtcolor[band-1][0],
                marker_color = mtcolor[band-1][1],
                error_y = dict(type='data', array=light_curve.sigmapsf_corr.values[idx], visible=True),
                marker = dict(size=10, opacity=0.5, showscale=False, line=dict(width=1)), 
                showlegend = True,
                legendgroup = mtcolor[band-1][0],
                hovertemplate ='<b>' + mtcolor[band-1][0] + ':\t\t</b>' + light_curve.magpsf_corr[idx].map('{:.3f}'.format).apply(str) + ' ± ' +
                                light_curve.sigmapsf_corr[idx].map('{:.3f}'.format).apply(str)  + '<br>' + 
                                '<b>MJD:\t</b>'+ light_curve.mjd[idx].apply(str)  + '<br>' + 
                                '<b>Date:\t</b>'+ [temp.strftime('%a, %d %b %Y %H:%M:%S UTC') for temp in Time(light_curve.mjd[idx].values, format='mjd').datetime]+ '<br>' +
                                '<extra></extra>'
                ))
    # fig['layout']['yaxis']['autorange'] = "reversed"
    fig.update_xaxes(
        tickangle=0,
        title_text="Modified Julian Dates [MJD]",
        title_font={"size": 14})
    
    fig.update_yaxes(
        autorange= 'reversed',
        tickangle=0,
        title_text="Magnitude [Mag]",
        title_font={"size": 14})
    
    fig.update_layout(
        title={'text':'<b>Light Curve</b>' + '<br>' +
                        '<a href="https://alerce.online/object/'+ event + '">Object: ' + event + 'text</a>',
                'font':dict(size=14),'x':0.5, 'y':0.9,'xanchor': 'center','yanchor': 'top'})
    
    if_show_fig = lambda show: fig.show() if show == True else False
    if_show_fig(show)


     

In [11]:
whos

Variable               Type                          Data/Info
--------------------------------------------------------------
GraphicLightCurve      function                      <function GraphicLightCurve at 0x7f23a93a8680>
Time                   ABCMeta                       <class 'astropy.time.core.Time'>
all_event              ndarray                       173693: 173693 elems, type `object`, 1389544 bytes (1.3251724243164062 Mb)
data                   DataFrame                                        oid  f<...>3844917 rows x 7 columns]
default_timer          builtin_function_or_method    <built-in function perf_counter>
drawGraphic            function                      <function drawGraphic at 0x7f23a93a8320>
drive                  module                        <module 'google.colab.dri<...>s/google/colab/drive.py'>
getStatistic           function                      <function getStatistic at 0x7f23a93a8200>
go                     module                        <module 'plotl

In [35]:
# parametros iniciales 
num_periods = 4
nbines = 10
array_periods = 2**(np.arange(num_periods)+0)
print(f'Arreglo de periodos: {array_periods}')

# cantidad de eventos a analizar 
limit = 5

# arreglo que contiene el tiempo de ejecucion de cada evento, posteriormente se obtendra el promedio
all_time_event = []

Arreglo de periodos: [1 2 4 8]


In [ ]:
# tiempo de ejecucion del algoritmo 
start_analyze = default_timer()

# all_data = []

for index, event in enumerate(all_event):
  # inicio - tiempo de ejecucion del analisis de cada evento
  start_time_event = default_timer()
  print(f'n:{index} - oid:{event}')

  # extrae todas las magnitudes asociadas al evento y las ordena por tiempo (mjd)
  light_curve = data.loc[data['oid'] == event].sort_values(by=['mjd'])
  

  # indices asociados a la banda 1 
  idx_banda = light_curve['fid'] == 1
  # light_curve['fid'].head()
  print(light_curve['fid'])
  tiempoNormalizado = (light_curve.mjd.values - light_curve.mjd.values.min())/(light_curve.mjd.values.max()- light_curve.mjd.values.min())
  print('tiempoNormalizado',len(tiempoNormalizado))
  tensor = np.array(variantion_period_v2(array_periods, tiempoNormalizado[idx_banda], light_curve.magpsf_corr[idx_banda].values, nbines))

  # print(f'Tensor {index}: \n{tensor}')
  fig = go.Figure()


  # graficar serie de tiempo con link a sitio de ALeRCE
  for band in light_curve['fid'].unique():
    GraphicLightCurve(fig,band,light_curve,True)
    

  # fin
  end_time_event = default_timer()
  # obtener el tiempo de ejecucion
  time_method_event = end_time_event - start_time_event
  # agrego el tiempo de ejecicion al arreglo para obtener el promedio general
  all_time_event.append(time_method_event)
  # print(f'tiempo de ejecucion por evento: {time_method_event}')
  print('-'*100)

  if (index == limit - 1):
    break


end_analyze = default_timer()
time_method_analyze = end_analyze - start_analyze
print('-'*100)
print(f'tiempo de ejecucion total: {time_method_analyze}')
print(f'promedio de tiempo de ejecucion por evento',np.mean(all_time_event))
print('-'*100)



# Analisis con distintos filtros


In [58]:
# parametros iniciales 
num_periods = 4
nbines = 10
array_periods = 2**(np.arange(num_periods)+0)
print(f'Arreglo de periodos: {array_periods}')

# cantidad de eventos a analizar 
limit = 5

# arreglo que contiene el tiempo de ejecucion de cada evento, posteriormente se obtendra el promedios
all_time_event = []

Arreglo de periodos: [1 2 4 8]


In [64]:
# tiempo de ejecucion del algoritmo 
start_analyze = default_timer()

for index, event in enumerate(all_event):
  # inicio - tiempo de ejecucion del analisis de cada evento
  start_time_event = default_timer()
  print(f'n:{index} - oid:{event}')
  fig = go.Figure()


  # extrae todas las magnitudes asociadas al evento y las ordena por tiempo (mjd)
  light_curve = data.loc[data['oid'] == event].sort_values(by=['mjd'])
  # mtcolor = [['g','green'],['r','red'],['b','blue']]

  print(light_curve['fid'])


  # graficar serie de tiempo con link a sitio de ALeRCE
  for band in light_curve['fid'].unique():
    GraphicLightCurve(fig,band,light_curve,False)

  
  # tensor que reune todos los filtros
  tensor_all_filter = []
  
  # nfid = numero del filter id (banda) = 1,green / 2,red
  for nfid in range(1,4):
    # type_filter = lambda nfid: 'Green' if nfid == 1 else 'Red'
    print('Filtro',nfid)
    idx_banda = light_curve['fid'] == nfid
    print(idx_banda)

    print(len(idx_banda))

    mjd_min = light_curve.mjd.values.min()

    tiempoNormalizado = (light_curve.mjd.values - mjd_min) / (light_curve.mjd.values.max() - mjd_min)
    # print('tiempoNormalizado\n',tiempoNormalizado)
    tensor = np.array(variantion_period_v2(array_periods, tiempoNormalizado[idx_banda], light_curve.magpsf_corr[idx_banda].values, nbines))
    tensor_all_filter.append(tensor)
    print(f'dimensiones {tensor.shape}')
    # print(f'Tensor {index}: \n{tensor}')
    print('-'*50)


  # asignar formato de tensor y muestralo
  tensor_all_filter = np.array(tensor_all_filter)
  print(f'dimensiones total {tensor_all_filter.shape}')
  # print(f'Tensor total {index}: \n{tensor_all_filter}')

  # fin
  end_time_event = default_timer()

  # obtener el tiempo de ejecucion
  time_method_event = end_time_event - start_time_event
  # agrego el tiempo de ejecicion al arreglo para obtener el promedio general
  all_time_event.append(time_method_event)

  print(f'tiempo de ejecucion por evento: {time_method_event}')
  print('-'*100)

  if (index == limit - 1):
    break


end_analyze = default_timer()
time_method_analyze = end_analyze - start_analyze
print('-'*100)
print(f'tiempo de ejecucion total: {time_method_analyze}')
print(f'promedio de tiempo de ejecucion por evento',np.mean(all_time_event))
print('-'*100)



n:0 - oid:ZTF17aaacfxu
4    1
0    1
1    1
2    1
3    1
Name: fid, dtype: int64
Filtro 1
4    True
0    True
1    True
2    True
3    True
Name: fid, dtype: bool
5
dimensiones (4, 4, 10)
--------------------------------------------------
Filtro 2
4    False
0    False
1    False
2    False
3    False
Name: fid, dtype: bool
5
dimensiones (4, 4, 10)
--------------------------------------------------
Filtro 3
4    False
0    False
1    False
2    False
3    False
Name: fid, dtype: bool
5
dimensiones (4, 4, 10)
--------------------------------------------------
dimensiones total (3, 4, 4, 10)
tiempo de ejecucion por evento: 0.4542175999995379
----------------------------------------------------------------------------------------------------
n:1 - oid:ZTF17aaaczwj
5    2
Name: fid, dtype: int64
Filtro 1
5    False
Name: fid, dtype: bool
1
dimensiones (4, 4, 10)
--------------------------------------------------
Filtro 2
5    True
Name: fid, dtype: bool
1
dimensiones (4, 4, 10)
----------

C:\Users\Leonardo Peña\AppData\Local\Temp\ipykernel_12020\2867386254.py:37: RuntimeWarning:

invalid value encountered in true_divide



10    1
45    1
29    1
58    2
46    1
     ..
91    2
19    1
38    1
71    2
85    2
Name: fid, Length: 88, dtype: int64
Filtro 1
10     True
45     True
29     True
58    False
46     True
      ...  
91    False
19     True
38     True
71    False
85    False
Name: fid, Length: 88, dtype: bool
88
dimensiones (4, 4, 10)
--------------------------------------------------
Filtro 2
10    False
45    False
29    False
58     True
46    False
      ...  
91     True
19    False
38    False
71     True
85     True
Name: fid, Length: 88, dtype: bool
88
dimensiones (4, 4, 10)
--------------------------------------------------
Filtro 3
10    False
45    False
29    False
58    False
46    False
      ...  
91    False
19    False
38    False
71    False
85    False
Name: fid, Length: 88, dtype: bool
88
dimensiones (4, 4, 10)
--------------------------------------------------
dimensiones total (3, 4, 4, 10)
tiempo de ejecucion por evento: 0.47344840000005206
---------------------------------